In [24]:
#!pip install mediapipe-model-maker

In [25]:

import os
import pathlib
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

In [26]:
path=str(pathlib.Path().resolve())+"/"

In [ ]:
from genericpath import isfile
from os import listdir, mkdir
import shutil

if not os.path.exists(path+"Piedra_Papel_Tijera_Formated"):
    gestos=([],[],[],[])
    directories = [f for f in listdir(path+"Piedra_Papel_Tijeras")]
    for i in directories:
        for index,(dirpath, dirnames, filenames) in enumerate(os.walk(path+"/Piedra_Papel_Tijeras/"+i+"/")):
            if len(dirnames)==0:
                for filename in filenames:
                    gestos[index-1].append(dirpath+"/"+filename)
    #mkdir(path+"Piedra_Papel_Tijera_Formated")
    #mkdir(path+"Piedra_Papel_Tijera_Formated/None")
    #mkdir(path+"Piedra_Papel_Tijera_Formated/Piedra")
    #mkdir(path+"Piedra_Papel_Tijera_Formated/Papel")
    #mkdir(path+"Piedra_Papel_Tijera_Formated/Tijera")
    for index,i in enumerate(gestos):
        for j in i:
            shutil.copy(j,path+"Piedra_Papel_Tijeras/"+i[index].split("/")[-2])


In [30]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

ValueError: Label set does not contain label "None".